In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# import python libraries
import numpy as np
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout, GRU, RNN
from tensorflow.keras.utils import to_categorical
from random import randint
import re

In [3]:
import nltk   # natural language tool kit library
nltk.download('gutenberg')  # downloads a library that NLTK uses

from nltk.corpus import gutenberg as gut  # downloads the gutenberg dataset
print(gut.fileids())  


[nltk_data] Downloading package gutenberg to /usr/share/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [4]:
# get the book text
book_text = nltk.corpus.gutenberg.raw('blake-poems.txt')

In [4]:
# print the first 500 characters of the text so we can look at it
print(book_text[:500])

[Poems by William Blake 1789]

 
SONGS OF INNOCENCE AND OF EXPERIENCE
and THE BOOK of THEL


 SONGS OF INNOCENCE
 
 
 INTRODUCTION
 
 Piping down the valleys wild,
   Piping songs of pleasant glee,
 On a cloud I saw a child,
   And he laughing said to me:
 
 "Pipe a song about a Lamb!"
   So I piped with merry cheer.
 "Piper, pipe that song again;"
   So I piped: he wept to hear.
 
 "Drop thy pipe, thy happy pipe;
   Sing thy songs of happy cheer:!"
 So I sang the same again,
   While he wept wi


In [5]:
def preprocess_text(sen):
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sen)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence.lower()

In [8]:
book_text = preprocess_text(book_text)


In [7]:
print(len(book_text))


34028


In [9]:
from nltk.tokenize import word_tokenize
# punkt is a sentence tokenizer that nltk requires. 
# It divides a text into a list of sentences, by using an unsupervised algorithm 
# to build a model for abbreviation words, collocations, and words that start sentences
nltk.download('punkt')

book_text_words = (word_tokenize(book_text))
n_words = len(book_text_words)
unique_words = len(set(book_text_words))

print('Total Words: %d' % n_words)
print('Unique Words: %d' % unique_words)


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Total Words: 6586
Unique Words: 1504


In [10]:
# convert words to numbers
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=unique_words)
tokenizer.fit_on_texts(book_text_words)

In [11]:
vocab_size = len(tokenizer.word_index) + 1    # word_index is the dictionary. Store the number of unique words in vocab_size variable
word_2_index = tokenizer.word_index           # store the dictionary in the variable called word_2_index

In [11]:
# just for exploration, let's print the 1000th word in the dictionary and it's index
print(book_text_words[1000])
print(word_2_index[book_text_words[500]])

did
363


In [12]:
input_sequence_words = []  # input sequences in words (used for metric evaluation later on)
input_sequence = []   # empty list to hold the sequences that will be input into our model
output_words = []     # empty list to hold the output words
input_seq_length = 100  # length of the input sequence

# form the input sequence list and the output words list
for i in range(0, n_words - input_seq_length , 1):
    in_seq = book_text_words[i:i + input_seq_length]
    input_sequence_words.append(in_seq)
    out_seq = book_text_words[i + input_seq_length]
    input_sequence.append([word_2_index[word] for word in in_seq])
    output_words.append(word_2_index[out_seq])

In [13]:
# print the first sequence to see what it looks like - a list of 100 integers that represent the first observation of words
print(len(input_sequence))      # print the number of input sequences
print(input_sequence[0])        # print the first input sequence
print(len(input_sequence[0]))   # print the length of the first input sequence

6490
[709, 40, 459, 276, 164, 3, 347, 2, 3, 348, 2, 1, 189, 3, 63, 164, 3, 347, 460, 349, 49, 1, 277, 121, 349, 164, 3, 226, 710, 12, 73, 165, 54, 2, 9, 227, 143, 5, 21, 278, 112, 711, 74, 45, 461, 7, 122, 462, 463, 278, 16, 112, 350, 45, 461, 9, 123, 5, 64, 712, 27, 278, 27, 50, 278, 98, 27, 164, 3, 50, 462, 45, 464, 1, 713, 350, 81, 9, 123, 7, 37, 5, 64, 463, 89, 15, 49, 2, 714, 4, 189, 16, 17, 190, 465, 45, 9, 279, 25, 6]
100


In [13]:
# reshape the input sequences to be 3-dimensional
#X = np.reshape(input_sequence, (3562, 100, 1))    # number of input sequences, length of each sequence
X = np.reshape(input_sequence, (len(input_sequence), input_seq_length, 1))

# Normalise the data by dividing by the max number of unique words (the vocab size)
X = X / float(vocab_size)

# one-hot encode the output words so that they can be used by the model (converts the output to 2-dimensions)
y = to_categorical(output_words)

In [15]:
print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (6490, 100, 1)
y shape: (6490, 1506)


In [16]:
model = Sequential()
# LSTM layer has 800 neurons (units).  The input shape is (100, 1) (Number of words in a sequence, 1 to make it 2D data) (Number of time-steps, features per time-step)
model.add(LSTM(800, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(LSTM(800, return_sequences=True))
model.add(LSTM(800))
model.add(Dense(y.shape[1], activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100, 800)          2566400   
                                                                 
 lstm_1 (LSTM)               (None, 100, 800)          5123200   
                                                                 
 lstm_2 (LSTM)               (None, 800)               5123200   
                                                                 
 dense (Dense)               (None, 1506)              1206306   
                                                                 
Total params: 14,019,106
Trainable params: 14,019,106
Non-trainable params: 0
_________________________________________________________________


In [17]:
# the output word can be one of any of the unique words in the vocabulary
# This means it is a multi-class calssification problem and we use the categorical crossentropy loss function
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
model.fit(X, y, batch_size=50, epochs=100, verbose=1)

Epoch 1/100
130/130 [==============================] - 905s 7s/step - loss: 6.5015
Epoch 2/100
130/130 [==============================] - 900s 7s/step - loss: 6.2230
Epoch 3/100
130/130 [==============================] - 919s 7s/step - loss: 6.1894
Epoch 4/100
130/130 [==============================] - 915s 7s/step - loss: 6.1806
Epoch 5/100
130/130 [==============================] - 929s 7s/step - loss: 6.1723
Epoch 6/100
130/130 [==============================] - 924s 7s/step - loss: 6.1696
Epoch 7/100
130/130 [==============================] - 823s 6s/step - loss: 6.1670
Epoch 8/100
130/130 [==============================] - 792s 6s/step - loss: 6.1659
Epoch 9/100
130/130 [==============================] - 785s 6s/step - loss: 6.1632
Epoch 10/100
130/130 [==============================] - 781s 6s/step - loss: 6.1606
Epoch 11/100
130/130 [==============================] - 788s 6s/step - loss: 6.1593
Epoch 12/100
130/130 [==============================] - 787s 6s/step - loss: 6.1590
E

In [14]:
model1 = Sequential()
# LSTM layer has 800 neurons (units).  The input shape is (100, 1) (Number of words in a sequence, 1 to make it 2D data) (Number of time-steps, features per time-step)
model1.add(LSTM(128, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model1.add(LSTM(128, return_sequences=True))
model1.add(LSTM(64))
model1.add(Dense(y.shape[1], activation='softmax'))

model1.summary()



model1.compile(loss='categorical_crossentropy', optimizer='adam')
print(model1.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100, 128)          66560     
                                                                 
 lstm_1 (LSTM)               (None, 100, 128)          131584    
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 1505)              97825     
                                                                 
Total params: 345,377
Trainable params: 345,377
Non-trainable params: 0
_________________________________________________________________
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (

In [15]:
model1.fit(X, y, batch_size=128, epochs=20, verbose=1)

Epoch 1/20
51/51 [==============================] - 28s 441ms/step - loss: 6.7311
Epoch 2/20
51/51 [==============================] - 23s 442ms/step - loss: 6.1978
Epoch 3/20
51/51 [==============================] - 23s 458ms/step - loss: 6.1742
Epoch 4/20
51/51 [==============================] - 23s 445ms/step - loss: 6.1663
Epoch 5/20
51/51 [==============================] - 23s 452ms/step - loss: 6.1637
Epoch 6/20
51/51 [==============================] - 23s 450ms/step - loss: 6.1626
Epoch 7/20
51/51 [==============================] - 22s 437ms/step - loss: 6.1615
Epoch 8/20
51/51 [==============================] - 23s 444ms/step - loss: 6.1606
Epoch 9/20
51/51 [==============================] - 23s 447ms/step - loss: 6.1584
Epoch 10/20
51/51 [==============================] - 23s 451ms/step - loss: 6.1576
Epoch 11/20
51/51 [==============================] - 24s 465ms/step - loss: 6.1556
Epoch 12/20
51/51 [==============================] - 23s 452ms/step - loss: 6.1555
Epoch 13/20
5

In [16]:
# Make Predictions
random_seq_index = np.random.randint(0, len(input_sequence)-1)    # select a random number from within the range of the number of input sequences
random_seq = input_sequence[random_seq_index]                     # get the input sequence that occurs at the randomly selected index (this is a list of integers)

index_2_word = dict(map(reversed, word_2_index.items())) # convert the integer sequence to its words
seed_word_sequence = [index_2_word[value] for value in random_seq]  # get the list of words that correspond to the integers in the randomly picked sequence

# join the words in the list and print the sequence of words
print(' '.join(seed_word_sequence))  # this prints the words from the randomly picked sequence that will be the seed for our prediction

# Predict next 100 words
word_sequence = []
for i in range(100):
    int_sample = np.reshape(random_seq, (1, len(random_seq), 1))    # reshape to make 3-D input (1 sequence, length of the sequence, 1 because the first LSTM requires another dimension)
    int_sample = int_sample / float(vocab_size)                     # normalise (as we normalised the training data)

    predicted_word_index = model1.predict(int_sample, verbose=0)     # predict the next word.  An array of the probabilities for each word in the vocab is returned.
    predicted_word_id = np.argmax(predicted_word_index)             # get the index of the maximum value (they are categorical so the max value gives the word in the vocab with the highest probability)
    word_sequence.append(index_2_word[ predicted_word_id])          # get the predicted word by finding the word at the predicted index and add it to our predicted word sequence list

    random_seq.append(predicted_word_id)                            # append the predicted word index to the next seuqence to be input into the model predict method
    random_seq = random_seq[1:len(random_seq)]                      # remove the first element of the sequence so it now has the new word but is the same length.

# BLEU score
seq = [' '.join(w) for w in input_sequence_words]
from nltk.translate.bleu_score import sentence_bleu
reference = seq
candidate = ' '.join(word_sequence) # make the list of words into a string
score = sentence_bleu(reference, candidate)
print('Seed word sequence: %s'%(' '.join(seed_word_sequence)))
print('Predicted words: %s'%(candidate))
print('BLEU Score for predicted words: %s'%(score))


or think on him who bore thy name graze after thee and weep for washed in life river my bright mane for ever shall shine like the gold as guard er the fold spring sound the flute now it mute bird delight day and night nightingale in the dale lark in sky merrily merrily merrily to welcome in the year little boy full of joy little girl sweet and small cock does crow so do you merry voice infant noise merrily merrily to welcome in the year little lamb here am come and lick my white neck let me pull
Seed word sequence: or think on him who bore thy name graze after thee and weep for washed in life river my bright mane for ever shall shine like the gold as guard er the fold spring sound the flute now it mute bird delight day and night nightingale in the dale lark in sky merrily merrily merrily to welcome in the year little boy full of joy little girl sweet and small cock does crow so do you merry voice infant noise merrily merrily to welcome in the year little lamb here am come and lick my w

In [18]:
model2 = Sequential()
# LSTM layer has 800 neurons (units).  The input shape is (100, 1) (Number of words in a sequence, 1 to make it 2D data) (Number of time-steps, features per time-step)
model2.add(LSTM(128, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model2.add(LSTM(128, return_sequences=True))
model2.add(LSTM(64))
model2.add(Dense(y.shape[1], activation='softmax'))

model2.summary()



model2.compile(loss='categorical_crossentropy', optimizer='adam')
print(model2.summary())
model2.fit(X, y, batch_size=128, epochs=50, verbose=1)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 100, 128)          66560     
                                                                 
 lstm_4 (LSTM)               (None, 100, 128)          131584    
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1505)              97825     
                                                                 
Total params: 345,377
Trainable params: 345,377
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)            

In [19]:
# Make Predictions
random_seq_index = np.random.randint(0, len(input_sequence)-1)    # select a random number from within the range of the number of input sequences
random_seq = input_sequence[random_seq_index]                     # get the input sequence that occurs at the randomly selected index (this is a list of integers)

index_2_word = dict(map(reversed, word_2_index.items())) # convert the integer sequence to its words
seed_word_sequence = [index_2_word[value] for value in random_seq]  # get the list of words that correspond to the integers in the randomly picked sequence

# join the words in the list and print the sequence of words
print(' '.join(seed_word_sequence))  # this prints the words from the randomly picked sequence that will be the seed for our prediction

# Predict next 100 words
word_sequence = []
for i in range(100):
    int_sample = np.reshape(random_seq, (1, len(random_seq), 1))    # reshape to make 3-D input (1 sequence, length of the sequence, 1 because the first LSTM requires another dimension)
    int_sample = int_sample / float(vocab_size)                     # normalise (as we normalised the training data)

    predicted_word_index = model2.predict(int_sample, verbose=0)     # predict the next word.  An array of the probabilities for each word in the vocab is returned.
    predicted_word_id = np.argmax(predicted_word_index)             # get the index of the maximum value (they are categorical so the max value gives the word in the vocab with the highest probability)
    word_sequence.append(index_2_word[ predicted_word_id])          # get the predicted word by finding the word at the predicted index and add it to our predicted word sequence list

    random_seq.append(predicted_word_id)                            # append the predicted word index to the next seuqence to be input into the model predict method
    random_seq = random_seq[1:len(random_seq)]                      # remove the first element of the sequence so it now has the new word but is the same length.

# BLEU score
seq = [' '.join(w) for w in input_sequence_words]
from nltk.translate.bleu_score import sentence_bleu
reference = seq
candidate = ' '.join(word_sequence) # make the list of words into a string
score = sentence_bleu(reference, candidate)
print('Seed word sequence: %s'%(' '.join(seed_word_sequence)))
print('Predicted words: %s'%(candidate))
print('BLEU Score for predicted words: %s'%(score))

becomes an infant small he becomes man of woe he doth feel the sorrow too think not thou canst sigh sigh and thy maker is not by think not thou canst weep tear and thy maker is not year oh he gives to us his joy that our grief he may destroy till our grief is fled an gone he doth sit by us and moan songs of experience introduction hear the voice of the bard who present past and future sees whose ears have heard the holy word that walked among the ancient tree calling the lapsed soul and
Seed word sequence: becomes an infant small he becomes man of woe he doth feel the sorrow too think not thou canst sigh sigh and thy maker is not by think not thou canst weep tear and thy maker is not year oh he gives to us his joy that our grief he may destroy till our grief is fled an gone he doth sit by us and moan songs of experience introduction hear the voice of the bard who present past and future sees whose ears have heard the holy word that walked among the ancient tree calling the lapsed soul 

In [20]:
model2.fit(X, y, batch_size=50, epochs=100, verbose=1)

Epoch 1/100
130/130 [==============================] - 38s 289ms/step - loss: 6.1773
Epoch 2/100
130/130 [==============================] - 37s 285ms/step - loss: 6.1635
Epoch 3/100
130/130 [==============================] - 37s 287ms/step - loss: 6.1439
Epoch 4/100
130/130 [==============================] - 38s 294ms/step - loss: 6.1223
Epoch 5/100
130/130 [==============================] - 38s 294ms/step - loss: 6.0925
Epoch 6/100
130/130 [==============================] - 38s 293ms/step - loss: 6.0581
Epoch 7/100
130/130 [==============================] - 39s 299ms/step - loss: 6.0205
Epoch 8/100
130/130 [==============================] - 39s 297ms/step - loss: 5.9806
Epoch 9/100
130/130 [==============================] - 38s 295ms/step - loss: 5.9398
Epoch 10/100
130/130 [==============================] - 38s 293ms/step - loss: 5.9032
Epoch 11/100
130/130 [==============================] - 38s 294ms/step - loss: 5.8667
Epoch 12/100
130/130 [==============================] - 39s 299

In [21]:
# Make Predictions
random_seq_index = np.random.randint(0, len(input_sequence)-1)    # select a random number from within the range of the number of input sequences
random_seq = input_sequence[random_seq_index]                     # get the input sequence that occurs at the randomly selected index (this is a list of integers)

index_2_word = dict(map(reversed, word_2_index.items())) # convert the integer sequence to its words
seed_word_sequence = [index_2_word[value] for value in random_seq]  # get the list of words that correspond to the integers in the randomly picked sequence

# join the words in the list and print the sequence of words
print(' '.join(seed_word_sequence))  # this prints the words from the randomly picked sequence that will be the seed for our prediction

# Predict next 100 words
word_sequence = []
for i in range(100):
    int_sample = np.reshape(random_seq, (1, len(random_seq), 1))    # reshape to make 3-D input (1 sequence, length of the sequence, 1 because the first LSTM requires another dimension)
    int_sample = int_sample / float(vocab_size)                     # normalise (as we normalised the training data)

    predicted_word_index = model2.predict(int_sample, verbose=0)     # predict the next word.  An array of the probabilities for each word in the vocab is returned.
    predicted_word_id = np.argmax(predicted_word_index)             # get the index of the maximum value (they are categorical so the max value gives the word in the vocab with the highest probability)
    word_sequence.append(index_2_word[ predicted_word_id])          # get the predicted word by finding the word at the predicted index and add it to our predicted word sequence list

    random_seq.append(predicted_word_id)                            # append the predicted word index to the next seuqence to be input into the model predict method
    random_seq = random_seq[1:len(random_seq)]                      # remove the first element of the sequence so it now has the new word but is the same length.



ah gentle may lay me down and gentle rest my head and gentle sleep the sleep of death and gently hear the voice of him that walketh in the garden in the evening time the lilly of the valley breathing in the humble grass answerd the lovely maid and said am watry weed and am very small and love to dwell in lowly vales so weak the gilded butterfly scarce perches on my head yet am visited from heaven and he that smiles on all walks in the valley and each morn over me spreads his hand saying rejoice thou


In [22]:
# BLEU score
seq = [' '.join(w) for w in input_sequence_words]
from nltk.translate.bleu_score import sentence_bleu
reference = seq
candidate = ' '.join(word_sequence) # make the list of words into a string
score = sentence_bleu(reference, candidate)
print('Seed word sequence: %s'%(' '.join(seed_word_sequence)))
print('Predicted words: %s'%(candidate))
print('BLEU Score for predicted words: %s'%(score))


Seed word sequence: ah gentle may lay me down and gentle rest my head and gentle sleep the sleep of death and gently hear the voice of him that walketh in the garden in the evening time the lilly of the valley breathing in the humble grass answerd the lovely maid and said am watry weed and am very small and love to dwell in lowly vales so weak the gilded butterfly scarce perches on my head yet am visited from heaven and he that smiles on all walks in the valley and each morn over me spreads his hand saying rejoice thou
Predicted words: the meekness of the night the the springing and the the steeds and the night and the springing and the the steeds and the night and the springing and the the steeds and the night and the springing and the the steeds and the night and the springing and the the steeds and the night and the springing and the the steeds and the night and the springing and the the steeds and the night and the springing and the the steeds and the night and the springing and th

#model1 with different paramenters

In [30]:
from keras.layers import Reshape

model3 = Sequential()
model3.add(LSTM(128, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model3.add(LSTM(128, return_sequences=True))
model3.add(LSTM(64, return_sequences=True))
model3.add(Reshape((1, -1)))
model3.add(LSTM(64))
model3.add(Dense(y.shape[1], activation='softmax'))
model3.compile(loss='categorical_crossentropy', optimizer='adam')
print(model3.summary())


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_17 (LSTM)              (None, 100, 128)          66560     
                                                                 
 lstm_18 (LSTM)              (None, 100, 128)          131584    
                                                                 
 lstm_19 (LSTM)              (None, 100, 64)           49408     
                                                                 
 reshape (Reshape)           (None, 1, 6400)           0         
                                                                 
 lstm_20 (LSTM)              (None, 64)                1655040   
                                                                 
 dense_2 (Dense)             (None, 1505)              97825     
                                                                 
Total params: 2,000,417
Trainable params: 2,000,417
No

In [31]:
# Make Predictions
random_seq_index = np.random.randint(0, len(input_sequence)-1)    # select a random number from within the range of the number of input sequences
random_seq = input_sequence[random_seq_index]                     # get the input sequence that occurs at the randomly selected index (this is a list of integers)

index_2_word = dict(map(reversed, word_2_index.items())) # convert the integer sequence to its words
seed_word_sequence = [index_2_word[value] for value in random_seq]  # get the list of words that correspond to the integers in the randomly picked sequence

# join the words in the list and print the sequence of words
print(' '.join(seed_word_sequence))  # this prints the words from the randomly picked sequence that will be the seed for our prediction

# Predict next 100 words
word_sequence = []
for i in range(100):
    int_sample = np.reshape(random_seq, (1, len(random_seq), 1))    # reshape to make 3-D input (1 sequence, length of the sequence, 1 because the first LSTM requires another dimension)
    int_sample = int_sample / float(vocab_size)                     # normalise (as we normalised the training data)

    predicted_word_index = model3.predict(int_sample, verbose=0)     # predict the next word.  An array of the probabilities for each word in the vocab is returned.
    predicted_word_id = np.argmax(predicted_word_index)             # get the index of the maximum value (they are categorical so the max value gives the word in the vocab with the highest probability)
    word_sequence.append(index_2_word[ predicted_word_id])          # get the predicted word by finding the word at the predicted index and add it to our predicted word sequence list

    random_seq.append(predicted_word_id)                            # append the predicted word index to the next seuqence to be input into the model predict method
    random_seq = random_seq[1:len(random_seq)]   

more nothing remains maid tell thee when pass away it is to tenfold life to love to peace and raptures holy unseen descending weigh my light wings upon balmy flowers and court the fair eyed dew to take me to her shining tent the weeping virgin trembling kneels before the risen sun till we arise link in golden band and never part but walk united bearing food to all our tender flowers dost thou little cloud fear that am not like thee for walk through the vales of har and smell the sweetest flowers but feed not the little flowers


In [32]:
# BLEU score
seq = [' '.join(w) for w in input_sequence_words]
from nltk.translate.bleu_score import sentence_bleu
reference = seq
candidate = ' '.join(word_sequence) # make the list of words into a string
score = sentence_bleu(reference, candidate)
print('Seed word sequence: %s'%(' '.join(seed_word_sequence)))
print('Predicted words: %s'%(candidate))
print('BLEU Score for predicted words: %s'%(score))

Seed word sequence: more nothing remains maid tell thee when pass away it is to tenfold life to love to peace and raptures holy unseen descending weigh my light wings upon balmy flowers and court the fair eyed dew to take me to her shining tent the weeping virgin trembling kneels before the risen sun till we arise link in golden band and never part but walk united bearing food to all our tender flowers dost thou little cloud fear that am not like thee for walk through the vales of har and smell the sweetest flowers but feed not the little flowers
Predicted words: afar afar afar snare snare snare arm arm arm arm arm arm arm arm arm arm vagabond vagabond vagabond vagabond vagabond vagabond vagabond vagabond hears hears springing springing springing springing springing springing springing springing springing arm arm arm arm arm arm arm to to to to to snare to to to to snare snare snare snare snare snare snare snare snare snare snare snare snare snare snare snare snare snare snare snare sn

In [33]:
from keras.layers import Dropout

model4 = Sequential()
model4.add(LSTM(128, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model4.add(Dropout(0.2))
model4.add(LSTM(128, return_sequences=True))
model4.add(Dropout(0.2))
model4.add(LSTM(64))
model4.add(Dropout(0.2))
model4.add(Dense(y.shape[1], activation='softmax'))

model4.compile(loss='categorical_crossentropy', optimizer='adam')


In [34]:
# Make Predictions
random_seq_index = np.random.randint(0, len(input_sequence)-1)    # select a random number from within the range of the number of input sequences
random_seq = input_sequence[random_seq_index]                     # get the input sequence that occurs at the randomly selected index (this is a list of integers)

index_2_word = dict(map(reversed, word_2_index.items())) # convert the integer sequence to its words
seed_word_sequence = [index_2_word[value] for value in random_seq]  # get the list of words that correspond to the integers in the randomly picked sequence

# join the words in the list and print the sequence of words
print(' '.join(seed_word_sequence))  # this prints the words from the randomly picked sequence that will be the seed for our prediction

# Predict next 100 words
word_sequence = []
for i in range(100):
    int_sample = np.reshape(random_seq, (1, len(random_seq), 1))    # reshape to make 3-D input (1 sequence, length of the sequence, 1 because the first LSTM requires another dimension)
    int_sample = int_sample / float(vocab_size)                     # normalise (as we normalised the training data)

    predicted_word_index = model4.predict(int_sample, verbose=0)     # predict the next word.  An array of the probabilities for each word in the vocab is returned.
    predicted_word_id = np.argmax(predicted_word_index)             # get the index of the maximum value (they are categorical so the max value gives the word in the vocab with the highest probability)
    word_sequence.append(index_2_word[ predicted_word_id])          # get the predicted word by finding the word at the predicted index and add it to our predicted word sequence list

    random_seq.append(predicted_word_id)                            # append the predicted word index to the next seuqence to be input into the model predict method
    random_seq = random_seq[1:len(random_seq)] 

sorrow and care dismay how shall the summer arise in joy or the summer fruits appear or how shall we gather what griefs destroy or bless the mellowing year when the blasts of winter appear to terzah whate er is born of mortal birth must be consumed with the earth to rise from generation free then what have to do with thee the sexes sprang from shame and pride blown in the morn in evening died but mercy changed death into sleep the sexes rose to work and weep thou mother of my mortal part with cruelty didst mould my


In [35]:
# BLEU score
seq = [' '.join(w) for w in input_sequence_words]
from nltk.translate.bleu_score import sentence_bleu
reference = seq
candidate = ' '.join(word_sequence) # make the list of words into a string
score = sentence_bleu(reference, candidate)
print('Seed word sequence: %s'%(' '.join(seed_word_sequence)))
print('Predicted words: %s'%(candidate))
print('BLEU Score for predicted words: %s'%(score))

Seed word sequence: sorrow and care dismay how shall the summer arise in joy or the summer fruits appear or how shall we gather what griefs destroy or bless the mellowing year when the blasts of winter appear to terzah whate er is born of mortal birth must be consumed with the earth to rise from generation free then what have to do with thee the sexes sprang from shame and pride blown in the morn in evening died but mercy changed death into sleep the sexes rose to work and weep thou mother of my mortal part with cruelty didst mould my
Predicted words: curtain curtain arrow arrow arrow arrow arrow arrow arrow arrow arrow curtain curtain curtain curtain curtain curtain curtain curtain curtain curtain curtain curtain curtain curtain curtain curtain curtain curtain curtain curtain curtain curtain curtain curtain curtain curtain curtain curtain curtain curtain curtain curtain curtain curtain curtain curtain curtain curtain curtain curtain curtain curtain curtain curtain curtain curtain curt

In [36]:
from keras import regularizers

model5 = Sequential()
model5.add(LSTM(128, input_shape=(X.shape[1], X.shape[2]), return_sequences=True, kernel_regularizer=regularizers.l2(0.01)))
model5.add(LSTM(128, return_sequences=True, kernel_regularizer=regularizers.l2(0.01)))
model5.add(LSTM(64, kernel_regularizer=regularizers.l2(0.01)))
model5.add(Dense(y.shape[1], activation='softmax', kernel_regularizer=regularizers.l2(0.01)))


In [37]:
# Make Predictions
random_seq_index = np.random.randint(0, len(input_sequence)-1)    # select a random number from within the range of the number of input sequences
random_seq = input_sequence[random_seq_index]                     # get the input sequence that occurs at the randomly selected index (this is a list of integers)

index_2_word = dict(map(reversed, word_2_index.items())) # convert the integer sequence to its words
seed_word_sequence = [index_2_word[value] for value in random_seq]  # get the list of words that correspond to the integers in the randomly picked sequence

# join the words in the list and print the sequence of words
print(' '.join(seed_word_sequence))  # this prints the words from the randomly picked sequence that will be the seed for our prediction

# Predict next 100 words
word_sequence = []
for i in range(100):
    int_sample = np.reshape(random_seq, (1, len(random_seq), 1))    # reshape to make 3-D input (1 sequence, length of the sequence, 1 because the first LSTM requires another dimension)
    int_sample = int_sample / float(vocab_size)                     # normalise (as we normalised the training data)

    predicted_word_index = model4.predict(int_sample, verbose=0)     # predict the next word.  An array of the probabilities for each word in the vocab is returned.
    predicted_word_id = np.argmax(predicted_word_index)             # get the index of the maximum value (they are categorical so the max value gives the word in the vocab with the highest probability)
    word_sequence.append(index_2_word[ predicted_word_id])          # get the predicted word by finding the word at the predicted index and add it to our predicted word sequence list

    random_seq.append(predicted_word_id)                            # append the predicted word index to the next seuqence to be input into the model predict method
    random_seq = random_seq[1:len(random_seq)] 
# BLEU score
seq = [' '.join(w) for w in input_sequence_words]
from nltk.translate.bleu_score import sentence_bleu
reference = seq
candidate = ' '.join(word_sequence) # make the list of words into a string
score = sentence_bleu(reference, candidate)
print('Seed word sequence: %s'%(' '.join(seed_word_sequence)))
print('Predicted words: %s'%(candidate))
print('BLEU Score for predicted words: %s'%(score))

now like mighty wild they raise to heaven the voice of song or like harmonious thunderings the seats of heaven among beneath them sit the aged man wise guardians of the poor then cherish pity lest you drive an angel from your door night the sun descending in the west the evening star does shine the birds are silent in their nest and must seek for mine the moon like flower in heaven high bower with silent delight sits and smiles on the night farewell green fields and happy grove where flocks have ta en delight where lambs have nibbled
Seed word sequence: now like mighty wild they raise to heaven the voice of song or like harmonious thunderings the seats of heaven among beneath them sit the aged man wise guardians of the poor then cherish pity lest you drive an angel from your door night the sun descending in the west the evening star does shine the birds are silent in their nest and must seek for mine the moon like flower in heaven high bower with silent delight sits and smiles on the n

#model5

#model6

In [38]:
model5 = Sequential()
# LSTM layer has 800 neurons (units).  The input shape is (100, 1) (Number of words in a sequence, 1 to make it 2D data) (Number of time-steps, features per time-step)
model5.add(LSTM(150, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model5.add(LSTM(140, return_sequences=True))
model5.add(LSTM(128))
model5.add(Dense(y.shape[1], activation='softmax'))

model5.summary()



model5.compile(loss='categorical_crossentropy', optimizer='adam')
#print(model2.summary())
model5.fit(X, y, batch_size=50, epochs=50, verbose=1)

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_27 (LSTM)              (None, 100, 150)          91200     
                                                                 
 lstm_28 (LSTM)              (None, 100, 140)          162960    
                                                                 
 lstm_29 (LSTM)              (None, 128)               137728    
                                                                 
 dense_5 (Dense)             (None, 1505)              194145    
                                                                 
Total params: 586,033
Trainable params: 586,033
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
130/130 [==============================] - 52s 366ms/step - loss: 6.5416
Epoch 2/50
130/130 [==============================] - 49s 373ms/step - loss: 6.2055
Epoch 3/50
1

In [39]:
# Make Predictions
random_seq_index = np.random.randint(0, len(input_sequence)-1)    # select a random number from within the range of the number of input sequences
random_seq = input_sequence[random_seq_index]                     # get the input sequence that occurs at the randomly selected index (this is a list of integers)

index_2_word = dict(map(reversed, word_2_index.items())) # convert the integer sequence to its words
seed_word_sequence = [index_2_word[value] for value in random_seq]  # get the list of words that correspond to the integers in the randomly picked sequence

# join the words in the list and print the sequence of words
print(' '.join(seed_word_sequence))  # this prints the words from the randomly picked sequence that will be the seed for our prediction

# Predict next 100 words
word_sequence = []
for i in range(100):
    int_sample = np.reshape(random_seq, (1, len(random_seq), 1))    # reshape to make 3-D input (1 sequence, length of the sequence, 1 because the first LSTM requires another dimension)
    int_sample = int_sample / float(vocab_size)                     # normalise (as we normalised the training data)

    predicted_word_index = model5.predict(int_sample, verbose=0)     # predict the next word.  An array of the probabilities for each word in the vocab is returned.
    predicted_word_id = np.argmax(predicted_word_index)             # get the index of the maximum value (they are categorical so the max value gives the word in the vocab with the highest probability)
    word_sequence.append(index_2_word[ predicted_word_id])          # get the predicted word by finding the word at the predicted index and add it to our predicted word sequence list

    random_seq.append(predicted_word_id)                            # append the predicted word index to the next seuqence to be input into the model predict method
    random_seq = random_seq[1:len(random_seq)] 
# BLEU score
seq = [' '.join(w) for w in input_sequence_words]
from nltk.translate.bleu_score import sentence_bleu
reference = seq
candidate = ' '.join(word_sequence) # make the list of words into a string
score = sentence_bleu(reference, candidate)
print('Seed word sequence: %s'%(' '.join(seed_word_sequence)))
print('Predicted words: %s'%(candidate))
print('BLEU Score for predicted words: %s'%(score))

merrily to welcome in the year little boy full of joy little girl sweet and small cock does crow so do you merry voice infant noise merrily merrily to welcome in the year little lamb here am come and lick my white neck let me pull your soft wool let me kiss your soft face merrily merrily to welcome in the year nurse song when the voices of children are heard on the green and laughing is heard on the hill my heart is at rest within my breast and everything else is still then come home my children the
Seed word sequence: merrily to welcome in the year little boy full of joy little girl sweet and small cock does crow so do you merry voice infant noise merrily merrily to welcome in the year little lamb here am come and lick my white neck let me pull your soft wool let me kiss your soft face merrily merrily to welcome in the year nurse song when the voices of children are heard on the green and laughing is heard on the hill my heart is at rest within my breast and everything else is still t

In [ ]:
#GRU
#Third model

model = Sequential()
# GRU layer has 800 neurons (units). The input shape is (100, 1) (Number of words in a sequence, 1 to make it 2D data) (Number of time-steps, features per time-step)
model.add(GRU(800, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(GRU(800, return_sequences=True))
model.add(GRU(800))
model.add(Dense(y.shape[1], activation='softmax'))

model.summary()

# the output word can be one of any of the unique words in the vocabulary
# This means it is a multi-class calssification problem and we use the categorical crossentropy loss function
model.compile(loss='categorical_crossentropy', optimizer='adam')